### Cargado de datos

Este es el codigo con el que se van a cargar los datos, estos datos se van a cargar desde un JSON

In [106]:
import json

with open('../data/datos2.json') as file:
    data = json.load(file)

data

{'nodes': [{'nombre': 'Nodo1',
   'carrera': 10,
   'calle': 10,
   'semaforo': {'have': True, 'duration': 0.4},
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo4']},
  {'nombre': 'Nodo2',
   'carrera': 10,
   'calle': 20,
   'semaforo': {'have': True, 'duration': 1},
   'punto_de_interes': False,
   'conexiones': ['Nodo1', 'Nodo3', 'Nodo5']},
  {'nombre': 'Nodo3',
   'carrera': 10,
   'calle': 30,
   'semaforo': {'have': False},
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo6']},
  {'nombre': 'Nodo4',
   'carrera': 20,
   'calle': 10,
   'semaforo': {'have': True, 'duration': 1.5},
   'punto_de_interes': False,
   'conexiones': ['Nodo1', 'Nodo5', 'Nodo7']},
  {'nombre': 'Nodo5',
   'carrera': 20,
   'calle': 20,
   'semaforo': {'have': False},
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo4', 'Nodo6', 'Nodo8']},
  {'nombre': 'Nodo6',
   'carrera': 20,
   'calle': 30,
   'semaforo': {'have': True, 'duration': 0.9},
   'punto_de_interes': False

### Objeto que se tiene que crear

El objeto Nodo o punto que se tiene que crear y con el metodo para cargar datos y crear un arreglo de objetos

In [107]:
class Nodo:
    def __init__(self, nombre, carrera, calle, semaforo, punto_de_interes, conexiones):
        self.nombre = nombre
        self.carrera = carrera
        self.calle = calle
        self.semaforo = semaforo
        self.punto_de_interes = punto_de_interes
        self.conexiones = conexiones
        self.g = 0
        self.h = 0
        self.f = 0
        self.parent = None

    def __str__(self):
        return (f"Nodo: {self.nombre}\n"
                f"Ubicación: Carrera {self.carrera}, Calle {self.calle}\n"
                f"Semaforo: {'Sí' if self.semaforo else 'No'}\n"
                f"Punto de Interés: {self.punto_de_interes}\n"
                f"Conexiones: {', '.join(con.nombre for con in self.conexiones)}\n"
                f"g: {self.g}, h: {self.h}, f: {self.f}\n"
                f"Parent: {self.parent.nombre if self.parent else 'Ninguno'}")

    """
    """
    def __eq__(self, other):
        return self.nombre == other.nombre
    def __hash__(self):
        return hash(self.nombre)


    def load_nodes(data):
        nodes = {}
        # Diccionario para mapear los nombres de nodos a objetos Nodo 
        node_map = {}
        for nodo_data in data["nodes"]:
            nodo = Nodo(
                nodo_data["nombre"],
                nodo_data["carrera"],
                nodo_data["calle"],
                nodo_data["semaforo"],
                nodo_data["punto_de_interes"],
                [],
            )
            nodes[nodo.nombre] = nodo
            # Agregamos el nodo al mapeo
            node_map[nodo.nombre] = nodo_data["conexiones"]
        # Se agregan las conexiones reales
        for k, v in nodes.items():
            v.conexiones = [nodes[conexion] for conexion in node_map[k]]
        return nodes

In [108]:
# Este seria el ejemplo de uso para saber si se cargaron bien los datos

# Arreglos de nodos (Ya nuestros datos cargados y convertidos en objetos de datos)
nodes_dict = Nodo.load_nodes(data=data)

# Para verificar que todo cargo bien hacemos un for para verificar dentro del array
for k, v in nodes_dict.items():
    print(f"{k}: \n{v}")
    # print([n.nombre for n in v.conexiones])

Nodo1: 
Nodo: Nodo1
Ubicación: Carrera 10, Calle 10
Semaforo: Sí
Punto de Interés: True
Conexiones: Nodo2, Nodo4
g: 0, h: 0, f: 0
Parent: Ninguno
Nodo2: 
Nodo: Nodo2
Ubicación: Carrera 10, Calle 20
Semaforo: Sí
Punto de Interés: False
Conexiones: Nodo1, Nodo3, Nodo5
g: 0, h: 0, f: 0
Parent: Ninguno
Nodo3: 
Nodo: Nodo3
Ubicación: Carrera 10, Calle 30
Semaforo: Sí
Punto de Interés: True
Conexiones: Nodo2, Nodo6
g: 0, h: 0, f: 0
Parent: Ninguno
Nodo4: 
Nodo: Nodo4
Ubicación: Carrera 20, Calle 10
Semaforo: Sí
Punto de Interés: False
Conexiones: Nodo1, Nodo5, Nodo7
g: 0, h: 0, f: 0
Parent: Ninguno
Nodo5: 
Nodo: Nodo5
Ubicación: Carrera 20, Calle 20
Semaforo: Sí
Punto de Interés: True
Conexiones: Nodo2, Nodo4, Nodo6, Nodo8
g: 0, h: 0, f: 0
Parent: Ninguno
Nodo6: 
Nodo: Nodo6
Ubicación: Carrera 20, Calle 30
Semaforo: Sí
Punto de Interés: False
Conexiones: Nodo3, Nodo5, Nodo9
g: 0, h: 0, f: 0
Parent: Ninguno
Nodo7: 
Nodo: Nodo7
Ubicación: Carrera 30, Calle 10
Semaforo: Sí
Punto de Interés: Tru

In [109]:
"""import heapq

def calculate_heuristic(node, end):
    return abs(node.calle - end.calle) + abs(node.carrera - end.carrera)


def astar(nodes_dict, start, end):
    open_list = []
    closed_list = set()
    
    start_node = nodes_dict[start]
    end_node = nodes_dict[end]

    #heapq.heappush(open_list, (0, start))

    while open_list:
        #current_node = heapq.heappop(open_list)[1]
        _, current_node = heapq.heappush(open_list, (0, start_node))
        closed_list.add(current_node)
        print(end)
        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.nombre)
                current_node = current_node.parent
            return path[::-1]

        for neighbor in current_node.conexiones:
            if neighbor in closed_list:
                continue
            print(neighbor)
            tentative_g = current_node.g + 10
            tentative_h = calculate_heuristic(neighbor, end_node)
            tentative_f = tentative_g + tentative_h
            #neighbor.g = current_node.g + 10
            #neighbor.h = calculate_heuristic(neighbor, end)
            #neighbor.f = neighbor.g + neighbor.h

            in_open_list = False
            for _, open_node in open_list:
                if neighbor == open_node:
                    in_open_list = True
                    if neighbor.g >= open_node.g:
                        break
            else:
                print("entro")
                neighbor.g = tentative_g
                neighbor.h = calculate_heuristic(neighbor, end_node)
                neighbor.f = tentative_f
                neighbor.parent = current_node
                if not in_open_list:
                    heapq.heappush(open_list, (neighbor.f, neighbor))

            #neighbor.parent = current_node
            #heapq.heappush(open_list, (neighbor.f, neighbor))

    return None"""

'import heapq\n\ndef calculate_heuristic(node, end):\n    return abs(node.calle - end.calle) + abs(node.carrera - end.carrera)\n\n\ndef astar(nodes_dict, start, end):\n    open_list = []\n    closed_list = set()\n    \n    start_node = nodes_dict[start]\n    end_node = nodes_dict[end]\n\n    #heapq.heappush(open_list, (0, start))\n\n    while open_list:\n        #current_node = heapq.heappop(open_list)[1]\n        _, current_node = heapq.heappush(open_list, (0, start_node))\n        closed_list.add(current_node)\n        print(end)\n        if current_node == end_node:\n            path = []\n            while current_node:\n                path.append(current_node.nombre)\n                current_node = current_node.parent\n            return path[::-1]\n\n        for neighbor in current_node.conexiones:\n            if neighbor in closed_list:\n                continue\n            print(neighbor)\n            tentative_g = current_node.g + 10\n            tentative_h = calculate_heu

In [110]:
import heapq

def calculate_heuristic(node, end):
    return abs(node.calle - end.calle) + abs(node.carrera - end.carrera)

def astar(nodes_dict, start_name, end_name):
    open_list = []
    closed_list = set()

    start_node = nodes_dict[start_name]
    end_node = nodes_dict[end_name]

    heapq.heappush(open_list, ( start_node.nombre, start_node))

    while open_list:
        _, current_node = heapq.heappop(open_list)
        closed_list.add(current_node.nombre)

        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.nombre)
                current_node = current_node.parent
            return path[::-1]

        for neighbor in current_node.conexiones:
            if neighbor.nombre in closed_list:
                continue

            tentative_g = current_node.g + 1
            tentative_f = tentative_g + calculate_heuristic(neighbor, end_node)

            in_open_list = False
            for item in open_list:
                _, open_node = item
                if neighbor.nombre == open_node.nombre:
                    in_open_list = True
                    if tentative_g >= neighbor.g:
                        break
            if not in_open_list or tentative_g < neighbor.g:
                neighbor.g = tentative_g
                neighbor.h = calculate_heuristic(neighbor, end_node)
                neighbor.f = tentative_f
                neighbor.parent = current_node
                if not in_open_list:
                    heapq.heappush(open_list, ( neighbor.nombre, neighbor))

    return None


In [111]:
path = astar(nodes_dict=nodes_dict, start_name='Nodo5', end_name='Nodo1')
print(path)

['Nodo5', 'Nodo2', 'Nodo1']
